In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
# General functions
def get_diff_row(df1, df2, on):
    df = df1.merge(df2, on=on, how='outer', indicator=True)
    return (df[df._merge!='both'])

def summarize_result(result_df):
    
    df = result_df.copy()
    # Summary
    def getSerotypeSummary(antigen, df):
        # get number of given serotypes
        total_s = df['given_'+antigen].notnull()
        num_total = df[total_s].shape[0]
        print("number of given_"+antigen+" serotypes is %d"
                %num_total)
        # get number of unpredicted serotype
        unpredicted_s = total_s & df['predicted_'+antigen].isnull()
        num_unpredicted = (df[unpredicted_s].shape[0])
        print("number of unpredicted_"+antigen+" serotypes is %d or %.2f%%"
                %(num_unpredicted, num_unpredicted/num_total*100))
        correct_s = total_s & ~unpredicted_s & (df['given_'+antigen]==df['predicted_'+antigen])
        correct_df = df[correct_s]
        num_correct = df[correct_s].shape[0]
        print("number of correctly predicted_"+antigen+" serotypes is %d or %.2f%%"
                %(num_correct, num_correct/num_total*100))
        incorrect_s = total_s & ~unpredicted_s & ~correct_s
        num_incorrect = df[incorrect_s].shape[0]
        print("number of incorrectly predicted_"+antigen+" serotypes is %d or %.2f%%"
                %(num_incorrect, num_incorrect/num_total*100))
        result_dfs = {
            'concordance': df[correct_s],
            'discrepancies': df[incorrect_s],
            'unpredicted': df[unpredicted_s]
        }
        return result_dfs, num_total, num_correct, num_incorrect
    result_dict = {}
    given_count = 0
    correct_count = 0
    incorrect_count = 0
    for antigen in ['O', 'H']:
        result_dfs, num_total, num_correct, num_incorrect = getSerotypeSummary(antigen, df)
        result_dict[antigen] = result_dfs
        given_count += num_total; correct_count+=num_correct;incorrect_count+=num_incorrect
    print("Overall concordance=%.2f%%(%d/%d)" %(correct_count/given_count*100, correct_count,given_count))
    print("Overall discrepancies=%.2f%%(%d/%d)" %(incorrect_count/given_count*100, incorrect_count, given_count))

    return result_dict

In [3]:
# Genbank only functions
def parse_assembly_summary(file):
    df = pd.read_csv(file, sep='\t', index_col='assembly_accession')
    
    # Get genome name
    df['genome_name'] = pd.DataFrame(df['ftp_path'].str.split('/', expand=True).iloc[:,-1])

    # Add O serotype info
    serotype_re = '(?<![A-z])(O\d{1,3})'
    df['given_O'] = df['organism_name'].str.extract(serotype_re, expand=False)
    df.given_O.fillna(df['infraspecific_name'].str.extract(serotype_re, expand=False), inplace=True)

    # Add H serotype info
    serotype_re2 = '\:(H\d{1,2})'
    df['given_H'] = df['organism_name'].str.extract(serotype_re2, expand=False)
    df.given_H.fillna(df['infraspecific_name'].str.extract(serotype_re2, expand=False), inplace=True)

    # Remove rows without serotype
    df = df[df['given_O'].notnull() | df['given_H'].notnull()].copy()
    
    # Filter out incorrect H serotypes
    filter_s = (df['given_H'].str[1:].astype(float)<=56) | df['given_H'].isnull()
    filter_s2 = (df['given_O'].str[1:].astype(float)<=186) | df['given_O'].isnull()
    df = df[filter_s & filter_s2]
    return df

In [4]:
# Enterobase only functions
def remove_blacklist_genomes(df):
    start_count = df.shape[0]
    print('Start with %d rows' %(start_count))
    df = df[~df.blacklisted]
    print('%d rows removed for blacklisted genome' %(start_count-df.shape[0]))
    print('End with %d rows' %(df.shape[0]))
    return df

def mark_blacklist_genomes(df, blacklist_file):
    blacklist_df = pd.read_csv(blacklist_file)
    df = df.merge(blacklist_df, on='genome_name', how='outer', indicator=True)
    df['blacklisted'] = df._merge!='left_only'
    df.drop('_merge', axis=1, inplace=True)
    return df

def read_info_df(file):
    # Read info df
    df = pd.read_csv(file).drop('Unnamed: 0', axis=1)
    df.columns = ['genome_name', 'given_H', 'given_O', 'serotype_tag']
    df.given_H='H'+df.loc[df.given_H.notnull()].given_H.astype(int).astype(str)
    df.given_O='O'+df.loc[df.given_O.notnull()].given_O.astype(int).astype(str)
    df = mark_blacklist_genomes(df, 'blacklist.csv')
    return df

def read_output_file(file):
    # Read from result file
    df = pd.read_csv(file)
    df = df[['index', 'O_prediction', 'O_info', 'H_prediction', 'H_info']]
    df.columns = ['genome_name', 'predicted_O', 'O_info', 'predicted_H', 'H_info']
    df.loc[df.predicted_O=='-', 'predicted_O'] = np.nan
    df.loc[df.predicted_H=='-', 'predicted_H'] = np.nan
    return df

def create_merge_df(info_file, output_file):
    # merge with info file
    df = read_output_file(output_file)
    df2 = read_info_df(info_file)
    df = df.merge(df2, on='genome_name', how='inner')
    df = df[['genome_name', 'given_O', 'predicted_O', 'O_info', 'given_H', 'predicted_H', 'H_info', 'serotype_tag', 'blacklisted']]
    return df

In [5]:
print("Genbank 97 97:")
mapping_df = parse_assembly_summary('assembly_summary.csv')[['genome_name', 'organism_name', 'infraspecific_name','given_O', 'given_H']]
output_df = pd.read_csv('output/genbank/output.csv')
output_df['genome_name']=output_df['index'].str.split('_genomic').str[0]
output_df.drop('index', axis=1, inplace=True)
output_df = output_df[['O_prediction', 'O_info', 'H_prediction', 'H_info', 'genome_name']]
output_df.columns = ['predicted_O', 'O_info', 'predicted_H', 'H_info', 'genome_name']
output_df.loc[output_df.predicted_O=='-', 'predicted_O'] = np.nan
output_df.loc[output_df.predicted_H=='-', 'predicted_H'] = np.nan
merge_df = mapping_df.merge(output_df, on='genome_name', how='inner')
results = summarize_result(merge_df)

Genbank 97 97:
number of given_O serotypes is 535
number of unpredicted_O serotypes is 26 or 4.86%
number of correctly predicted_O serotypes is 504 or 94.21%
number of incorrectly predicted_O serotypes is 5 or 0.93%
number of given_H serotypes is 387
number of unpredicted_H serotypes is 15 or 3.88%
number of correctly predicted_H serotypes is 368 or 95.09%
number of incorrectly predicted_H serotypes is 4 or 1.03%
Overall concordance=94.58%(872/922)
Overall discrepancies=0.98%(9/922)


In [6]:
print("Genbank 90 50:")
mapping_df = parse_assembly_summary('assembly_summary.csv')[['genome_name', 'organism_name', 'infraspecific_name','given_O', 'given_H']]
output_df = pd.read_csv('output/genbank9050/output.csv')
output_df['genome_name']=output_df['index'].str.split('_genomic').str[0]
output_df.drop('index', axis=1, inplace=True)
output_df = output_df[['O_prediction', 'O_info', 'H_prediction', 'H_info', 'genome_name']]
output_df.columns = ['predicted_O', 'O_info', 'predicted_H', 'H_info', 'genome_name']
output_df.loc[output_df.predicted_O=='-', 'predicted_O'] = np.nan
output_df.loc[output_df.predicted_H=='-', 'predicted_H'] = np.nan
merge_df = mapping_df.merge(output_df, on='genome_name', how='inner')
results = summarize_result(merge_df)

Genbank 90 50:
number of given_O serotypes is 535
number of unpredicted_O serotypes is 15 or 2.80%
number of correctly predicted_O serotypes is 515 or 96.26%
number of incorrectly predicted_O serotypes is 5 or 0.93%
number of given_H serotypes is 387
number of unpredicted_H serotypes is 2 or 0.52%
number of correctly predicted_H serotypes is 381 or 98.45%
number of incorrectly predicted_H serotypes is 4 or 1.03%
Overall concordance=97.18%(896/922)
Overall discrepancies=0.98%(9/922)


In [7]:
print("Enterobase 97 97:")
merge_df = create_merge_df('enterobase_serotype.csv', 'output/enterobase/output.csv')
merge_df = remove_blacklist_genomes(merge_df)
results = summarize_result(merge_df)

Enterobase 97 97:
Start with 5844 rows
489 rows removed for blacklisted genome
End with 5355 rows
number of given_O serotypes is 5223
number of unpredicted_O serotypes is 827 or 15.83%
number of correctly predicted_O serotypes is 4233 or 81.05%
number of incorrectly predicted_O serotypes is 163 or 3.12%
number of given_H serotypes is 2570
number of unpredicted_H serotypes is 44 or 1.71%
number of correctly predicted_H serotypes is 2384 or 92.76%
number of incorrectly predicted_H serotypes is 142 or 5.53%
Overall concordance=84.91%(6617/7793)
Overall discrepancies=3.91%(305/7793)


In [8]:
print("Enterobase 90 50:")
merge_df = create_merge_df('enterobase_serotype.csv', 'output/enterobase 9050/output.csv')
merge_df = remove_blacklist_genomes(merge_df)
results = summarize_result(merge_df)

Enterobase 90 50:
Start with 5844 rows
489 rows removed for blacklisted genome
End with 5355 rows
number of given_O serotypes is 5223
number of unpredicted_O serotypes is 532 or 10.19%
number of correctly predicted_O serotypes is 4474 or 85.66%
number of incorrectly predicted_O serotypes is 217 or 4.15%
number of given_H serotypes is 2570
number of unpredicted_H serotypes is 6 or 0.23%
number of correctly predicted_H serotypes is 2416 or 94.01%
number of incorrectly predicted_H serotypes is 148 or 5.76%
Overall concordance=88.41%(6890/7793)
Overall discrepancies=4.68%(365/7793)
